In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import os
from urllib import parse
import numpy as np

In [2]:

def step2_get_reple_href() :
    #영화 코드를 불러온다.
    # 테스트용
    code_list = ['181414', '92575','167638','155263']


    # 영화코드와 주소를 합쳐서 요청할 주소를 만든다.
    url_list = pd.DataFrame()
    for code in code_list:
        site = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=%s&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false' % code
        # print(site)
        url_list = url_list.append([[site]], ignore_index=True)

    url_list.columns = ['url']
    url_list.to_csv('movie_url_list.csv', index=False, encoding='utf-8-sig')
    print('저장완료')



In [3]:
def step3_get_reply_data() :
    # csv에 저장되어 있는 url 데이터를 가져온다.
    df = pd.read_csv('movie_url_list.csv')
    url_list = df['url'].tolist()

    # for url in url_list :
    #    print(url)
    # url = url_list[0]

    for url in url_list :
        print(url)
        # 해당 영화의 첫 페이지 html 데이터를 가져온다. (총 몇건의 리뷰가 있는지 확인해서 "페이지수"를 계산하기 위함)
        response = requests.get(url)
        bs = BeautifulSoup(response.content, 'html.parser')
        # print(bs)

        # 총 페이지 수를 구한다.
        strong = bs.select('.total em')
        #1->0
        score_total = int(strong[0].text.replace(',', ''))    # 쉼표 없애기 / int(정수형)로 만들기
        # print(score_total)
        pageCnt = score_total // 10     # 한페이지당 10개의 리뷰가 있어서
        if score_total % 10 > 0 :
            pageCnt += 1
        print(pageCnt)

        # 전체 페이지를 돌면서 140평 데이터를 가져온다.
        # 현재 페이지
        now_page = 1

#         pageCnt = 5        # 테스트로 5페이지까지만
        
        while now_page <= pageCnt :
            #sleep(1)
            #sleep(0.0001)
            # 요청할 페이지의 주소
            url2 = url + '&page=' + str(now_page)
            # print(url2)

            # 140자평 데이터를 추출한다.
            response2 = requests.get(url2)
            bs2 = BeautifulSoup(response2.content, 'html.parser')

            result_df = pd.DataFrame()

            # li 태그들을 가져온다.(score_reple 태그-리뷰-를 포함하고 있는)
            lis = bs2.select('.score_result li')

            for obj in lis :
                # 평점
                star_score = obj.select('.star_score em')[0].text
                # 140자평
                score_reple = obj.select('.score_reple p')[0].text
                # print(star_score)
                # print(score_reple)

                # 저장한다.
                result_df = result_df.append([[score_reple, star_score]], ignore_index=True)

            if os.path.exists('model_review.csv') == False :     # 아직 파일이 없으면 파일을 만든다.
                result_df.columns = ['text', 'score']
                result_df.to_csv('model_review.csv', index=False, encoding='utf-8-sig')
            else :                                               # 이미 파일이 있으면 결과를 더한다.
                result_df.to_csv('model_review.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

            print("%d / %d" % (now_page, pageCnt))               # 진행경과를 보여준다. n번째 중 몇 번째 진행중인지.
            now_page += 1

    print('저장완료')



In [4]:
# 140자평 데이터 전처리 함수
def text_preprocessing(text) :
    if text.startswith('관람객'):
        return text[3:]
    ######
    elif text.startswith('스포일러가 포함된 감상평입니다. 감상평 보기'):
        return text[23:]
    #####
    else :
        return text

# 평점 전처리 함수
def star_preprocessing(text) :
    value = int(text)

    if value <= 5 :
        return '0'
    else :
        return '1'




In [5]:
def step4_data_preprocessing() :
    # 수집한 데이터를 읽어온다.
    df = pd.read_csv('model_review.csv')
    print(len(df))
    # 전처리를 수행한다.
    df['text'] = df['text'].apply(text_preprocessing)
    df['score'] = df['score'].apply(star_preprocessing)
    
    # 학습데이터와 테스트 데이터로 나눈다.
    text_list = df['text'].tolist()
    star_list = df['score'].tolist()

    from sklearn.model_selection import train_test_split

    # 70%는 학습, 30%는 test
    text_train, text_test, star_train, star_test = train_test_split(text_list, star_list, test_size=0.3, random_state=0)

    return text_train, text_test, star_train, star_test



In [6]:
# step4_data_preprocessing()

In [7]:
from sklearn.model_selection import GridSearchCV #하이퍼파라미터 최적화
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle
from konlpy.tag import *

# 형태소 분석을 위한 함수
def tokenizer(text) :
    okt = Okt()
    return okt.morphs(text)
def using_GridSearch(X_train,y_train,tfidf,logistic):
    tfidf.fit(X_train)
    tfv_X_train= tfidf.transform(X_train)
    params = {'C': [1,3,5,7,9,11,13,15,100,200,300],
             'penalty':['l1','l2']}
    grid_cv = GridSearchCV(logistic, param_grid=params, cv =4, scoring='accuracy', verbose=1)
    grid_cv.fit(tfv_X_train, y_train)
    print(grid_cv.best_params_)    
    print(grid_cv.best_score_)
    
def step5_learning(X_train, y_train, X_test, y_test):

    # 주어진 데이터를 단어 사전으로 만들고 각 단어의 빈도수를 계산한 후 벡터화 하는 객체 생성
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer,min_df=3, max_df=0.9, ngram_range=(1,2)) #ngram (1,2) 
     
    # 문장별 나오는 단어수 세서 수치화, 벡터화해서 학습을 시킨다.
    logistic = LogisticRegression(C=100, penalty='l2', random_state=0)
    
    #최적의 파라미터 찾기
#     using_GridSearch(X_train,y_train,tfidf,logistic)
    
    
    pipe = Pipeline([('vect', tfidf), ('clf', logistic)])

    # 학습한다.
    pipe.fit(X_train, y_train)

    # 학습 정확도 측정
    y_pred = pipe.predict(X_test)
    print(accuracy_score(y_test, y_pred))

    # 학습된 모델을 저장한다.
    with open('pipe_model.dat', 'wb') as fp :
        pickle.dump(pipe, fp)
        
    print('저장완료')



In [8]:
# 스크래핑 함수
def scrapping() :

    # 각 영화 코드 데이터를 가져와 저장한다.
#     step1_get_detail_url()

    # 140자 평 데이터가 있는 페이지의 주소를 저장한다.
    step2_get_reple_href()

    # 140평 데이터를 가져온다.
    step3_get_reply_data()

# 학습 함수
def learing() :
    text_train, text_test, star_train, star_test = step4_data_preprocessing()
    step5_learning(text_train, star_train, text_test, star_test)


In [9]:
if __name__ == "__main__":
    scrapping()
    learing()

저장완료
https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=181414&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false
1077
1 / 1077
2 / 1077
3 / 1077
4 / 1077
5 / 1077
6 / 1077
7 / 1077
8 / 1077
9 / 1077
10 / 1077
11 / 1077
12 / 1077
13 / 1077
14 / 1077
15 / 1077
16 / 1077
17 / 1077
18 / 1077
19 / 1077
20 / 1077
21 / 1077
22 / 1077
23 / 1077
24 / 1077
25 / 1077
26 / 1077
27 / 1077
28 / 1077
29 / 1077
30 / 1077
31 / 1077
32 / 1077
33 / 1077
34 / 1077
35 / 1077
36 / 1077
37 / 1077
38 / 1077
39 / 1077
40 / 1077
41 / 1077
42 / 1077
43 / 1077
44 / 1077
45 / 1077
46 / 1077
47 / 1077
48 / 1077
49 / 1077
50 / 1077
51 / 1077
52 / 1077
53 / 1077
54 / 1077
55 / 1077
56 / 1077
57 / 1077
58 / 1077
59 / 1077
60 / 1077
61 / 1077
62 / 1077
63 / 1077
64 / 1077
65 / 1077
66 / 1077
67 / 1077
68 / 1077
69 / 1077
70 / 1077
71 / 1077
72 / 1077
73 / 1077
74 / 1077
75 / 1077
76 / 1077
77 / 1077
78 / 1077
79 / 1077
80 / 1077
81 / 1077
82 / 1

740 / 1077
741 / 1077
742 / 1077
743 / 1077
744 / 1077
745 / 1077
746 / 1077
747 / 1077
748 / 1077
749 / 1077
750 / 1077
751 / 1077
752 / 1077
753 / 1077
754 / 1077
755 / 1077
756 / 1077
757 / 1077
758 / 1077
759 / 1077
760 / 1077
761 / 1077
762 / 1077
763 / 1077
764 / 1077
765 / 1077
766 / 1077
767 / 1077
768 / 1077
769 / 1077
770 / 1077
771 / 1077
772 / 1077
773 / 1077
774 / 1077
775 / 1077
776 / 1077
777 / 1077
778 / 1077
779 / 1077
780 / 1077
781 / 1077
782 / 1077
783 / 1077
784 / 1077
785 / 1077
786 / 1077
787 / 1077
788 / 1077
789 / 1077
790 / 1077
791 / 1077
792 / 1077
793 / 1077
794 / 1077
795 / 1077
796 / 1077
797 / 1077
798 / 1077
799 / 1077
800 / 1077
801 / 1077
802 / 1077
803 / 1077
804 / 1077
805 / 1077
806 / 1077
807 / 1077
808 / 1077
809 / 1077
810 / 1077
811 / 1077
812 / 1077
813 / 1077
814 / 1077
815 / 1077
816 / 1077
817 / 1077
818 / 1077
819 / 1077
820 / 1077
821 / 1077
822 / 1077
823 / 1077
824 / 1077
825 / 1077
826 / 1077
827 / 1077
828 / 1077
829 / 1077
830 / 1077

394 / 3643
395 / 3643
396 / 3643
397 / 3643
398 / 3643
399 / 3643
400 / 3643
401 / 3643
402 / 3643
403 / 3643
404 / 3643
405 / 3643
406 / 3643
407 / 3643
408 / 3643
409 / 3643
410 / 3643
411 / 3643
412 / 3643
413 / 3643
414 / 3643
415 / 3643
416 / 3643
417 / 3643
418 / 3643
419 / 3643
420 / 3643
421 / 3643
422 / 3643
423 / 3643
424 / 3643
425 / 3643
426 / 3643
427 / 3643
428 / 3643
429 / 3643
430 / 3643
431 / 3643
432 / 3643
433 / 3643
434 / 3643
435 / 3643
436 / 3643
437 / 3643
438 / 3643
439 / 3643
440 / 3643
441 / 3643
442 / 3643
443 / 3643
444 / 3643
445 / 3643
446 / 3643
447 / 3643
448 / 3643
449 / 3643
450 / 3643
451 / 3643
452 / 3643
453 / 3643
454 / 3643
455 / 3643
456 / 3643
457 / 3643
458 / 3643
459 / 3643
460 / 3643
461 / 3643
462 / 3643
463 / 3643
464 / 3643
465 / 3643
466 / 3643
467 / 3643
468 / 3643
469 / 3643
470 / 3643
471 / 3643
472 / 3643
473 / 3643
474 / 3643
475 / 3643
476 / 3643
477 / 3643
478 / 3643
479 / 3643
480 / 3643
481 / 3643
482 / 3643
483 / 3643
484 / 3643

1129 / 3643
1130 / 3643
1131 / 3643
1132 / 3643
1133 / 3643
1134 / 3643
1135 / 3643
1136 / 3643
1137 / 3643
1138 / 3643
1139 / 3643
1140 / 3643
1141 / 3643
1142 / 3643
1143 / 3643
1144 / 3643
1145 / 3643
1146 / 3643
1147 / 3643
1148 / 3643
1149 / 3643
1150 / 3643
1151 / 3643
1152 / 3643
1153 / 3643
1154 / 3643
1155 / 3643
1156 / 3643
1157 / 3643
1158 / 3643
1159 / 3643
1160 / 3643
1161 / 3643
1162 / 3643
1163 / 3643
1164 / 3643
1165 / 3643
1166 / 3643
1167 / 3643
1168 / 3643
1169 / 3643
1170 / 3643
1171 / 3643
1172 / 3643
1173 / 3643
1174 / 3643
1175 / 3643
1176 / 3643
1177 / 3643
1178 / 3643
1179 / 3643
1180 / 3643
1181 / 3643
1182 / 3643
1183 / 3643
1184 / 3643
1185 / 3643
1186 / 3643
1187 / 3643
1188 / 3643
1189 / 3643
1190 / 3643
1191 / 3643
1192 / 3643
1193 / 3643
1194 / 3643
1195 / 3643
1196 / 3643
1197 / 3643
1198 / 3643
1199 / 3643
1200 / 3643
1201 / 3643
1202 / 3643
1203 / 3643
1204 / 3643
1205 / 3643
1206 / 3643
1207 / 3643
1208 / 3643
1209 / 3643
1210 / 3643
1211 / 3643
1212

1812 / 3643
1813 / 3643
1814 / 3643
1815 / 3643
1816 / 3643
1817 / 3643
1818 / 3643
1819 / 3643
1820 / 3643
1821 / 3643
1822 / 3643
1823 / 3643
1824 / 3643
1825 / 3643
1826 / 3643
1827 / 3643
1828 / 3643
1829 / 3643
1830 / 3643
1831 / 3643
1832 / 3643
1833 / 3643
1834 / 3643
1835 / 3643
1836 / 3643
1837 / 3643
1838 / 3643
1839 / 3643
1840 / 3643
1841 / 3643
1842 / 3643
1843 / 3643
1844 / 3643
1845 / 3643
1846 / 3643
1847 / 3643
1848 / 3643
1849 / 3643
1850 / 3643
1851 / 3643
1852 / 3643
1853 / 3643
1854 / 3643
1855 / 3643
1856 / 3643
1857 / 3643
1858 / 3643
1859 / 3643
1860 / 3643
1861 / 3643
1862 / 3643
1863 / 3643
1864 / 3643
1865 / 3643
1866 / 3643
1867 / 3643
1868 / 3643
1869 / 3643
1870 / 3643
1871 / 3643
1872 / 3643
1873 / 3643
1874 / 3643
1875 / 3643
1876 / 3643
1877 / 3643
1878 / 3643
1879 / 3643
1880 / 3643
1881 / 3643
1882 / 3643
1883 / 3643
1884 / 3643
1885 / 3643
1886 / 3643
1887 / 3643
1888 / 3643
1889 / 3643
1890 / 3643
1891 / 3643
1892 / 3643
1893 / 3643
1894 / 3643
1895

...

1663 / 2339
1664 / 2339
1665 / 2339
1666 / 2339
1667 / 2339
1668 / 2339
1669 / 2339
1670 / 2339
1671 / 2339
1672 / 2339
1673 / 2339
1674 / 2339
1675 / 2339
1676 / 2339
1677 / 2339
1678 / 2339
1679 / 2339
1680 / 2339
1681 / 2339
1682 / 2339
1683 / 2339
1684 / 2339
1685 / 2339
1686 / 2339
1687 / 2339
1688 / 2339
1689 / 2339
1690 / 2339
1691 / 2339
1692 / 2339
1693 / 2339
1694 / 2339
1695 / 2339
1696 / 2339
1697 / 2339
1698 / 2339
1699 / 2339
1700 / 2339
1701 / 2339
1702 / 2339
1703 / 2339
1704 / 2339
1705 / 2339
1706 / 2339
1707 / 2339
1708 / 2339
1709 / 2339
1710 / 2339
1711 / 2339
1712 / 2339
1713 / 2339
1714 / 2339
1715 / 2339
1716 / 2339
1717 / 2339
1718 / 2339
1719 / 2339
1720 / 2339
1721 / 2339
1722 / 2339
1723 / 2339
1724 / 2339
1725 / 2339
1726 / 2339
1727 / 2339
1728 / 2339
1729 / 2339
1730 / 2339
1731 / 2339
1732 / 2339
1733 / 2339
1734 / 2339
1735 / 2339
1736 / 2339
1737 / 2339
1738 / 2339
1739 / 2339
1740 / 2339
1741 / 2339
1742 / 2339
1743 / 2339
1744 / 2339
1745 / 2339
1746

2 / 881
3 / 881
4 / 881
5 / 881
6 / 881
7 / 881
8 / 881
9 / 881
10 / 881
11 / 881
12 / 881
13 / 881
14 / 881
15 / 881
16 / 881
17 / 881
18 / 881
19 / 881
20 / 881
21 / 881
22 / 881
23 / 881
24 / 881
25 / 881
26 / 881
27 / 881
28 / 881
29 / 881
30 / 881
31 / 881
32 / 881
33 / 881
34 / 881
35 / 881
36 / 881
37 / 881
38 / 881
39 / 881
40 / 881
41 / 881
42 / 881
43 / 881
44 / 881
45 / 881
46 / 881
47 / 881
48 / 881
49 / 881
50 / 881
51 / 881
52 / 881
53 / 881
54 / 881
55 / 881
56 / 881
57 / 881
58 / 881
59 / 881
60 / 881
61 / 881
62 / 881
63 / 881
64 / 881
65 / 881
66 / 881
67 / 881
68 / 881
69 / 881
70 / 881
71 / 881
72 / 881
73 / 881
74 / 881
75 / 881
76 / 881
77 / 881
78 / 881
79 / 881
80 / 881
81 / 881
82 / 881
83 / 881
84 / 881
85 / 881
86 / 881
87 / 881
88 / 881
89 / 881
90 / 881
91 / 881
92 / 881
93 / 881
94 / 881
95 / 881
96 / 881
97 / 881
98 / 881
99 / 881
100 / 881
101 / 881
102 / 881
103 / 881
104 / 881
105 / 881
106 / 881
107 / 881
108 / 881
109 / 881
110 / 881
111 / 881
112 / 

832 / 881
833 / 881
834 / 881
835 / 881
836 / 881
837 / 881
838 / 881
839 / 881
840 / 881
841 / 881
842 / 881
843 / 881
844 / 881
845 / 881
846 / 881
847 / 881
848 / 881
849 / 881
850 / 881
851 / 881
852 / 881
853 / 881
854 / 881
855 / 881
856 / 881
857 / 881
858 / 881
859 / 881
860 / 881
861 / 881
862 / 881
863 / 881
864 / 881
865 / 881
866 / 881
867 / 881
868 / 881
869 / 881
870 / 881
871 / 881
872 / 881
873 / 881
874 / 881
875 / 881
876 / 881
877 / 881
878 / 881
879 / 881
880 / 881
881 / 881
저장완료
79381


C:\Users\IIS3\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8761284904471971
저장완료
